In [613]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

import eli5

from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
from konlpy.tag import Mecab


# score 정보 -> 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
# score 정보 -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도.
# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature)

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification 처럼 summarization + 빈도수 처리 + ....
# N-gram 활용 + 시각화 잘하기

In [614]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [615]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [616]:
dict_pos=[]
dict_neg=[]
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
    dict_pos.append(line.split('\n')[0])
    
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
    dict_neg.append(line.split('\n')[0])

In [617]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [618]:
reviews1

{'병사보다 높은 자유도(부사관기준), 육군 보병 장교보다 원활한 업무(장교기준), 생각보다 편한 병사(병사기준)\r 조금 힘들더라도 빡쌔게 근무 할 수 있는 곳을 원한다면 이곳이다.': '34',
 '사서로써의 보람을 느낄 수 있으나 그 외에 모든 것들이 아쉬운 곳.': '34',
 '대대손손 척을 진 적이 입사한다고 해도 도시락 싸들고 가서 말릴 기관. 만약 내가 누군가에게 가스라이팅을 당하는 것을 즐긴다면 추천할 회사. 개쌍욕을 하루 종일 들어도 타격이 없다 하는  수준의 무감정자에게 추천하는 회사. 나는 인간임을 포기하고 싶고 그 존엄성을 바닥에 짓밟혀보고자 하는 분에게 특히나 추천드립니다.': '34',
 '멘토를 일하는 개로봄...임금을 늦게주고 임원진들은  멘토에게 관심이없다.': '34',
 '조직 구조 상 여러가지 한계점이 많아 다소 아쉬움이 남음.': '34',
 '신생기관이라 체계적이지 않은것이 장점지자 단점. 허술한게 많아 악용하면 얼마든지 악용가능해요.': '32',
 '꿈과 희망이 없다면 추천..월급루팡하기 좋은곳 \r자기개발은 확실히되거나(스스로 해야하니깐) 아니면 다른 기관으로 이직하고 나서 적응이 어려울것 같음.': '32',
 '누군가 썼듯이 원수에게도 추천하고 싶지.않은 전형적인 회사임': '32',
 '별 한개도 아깝다. 정말 최악의 직장. 출퇴근길이 퇴사 생각밖에 나지 않는 곳.': '30',
 '한줄로 정리하면 딱 이렇습니다. "공공기관이 맞나요?"': '30',
 '직장내 괴롭힘과 갑질 만연. 존중 없는 분위기. 중소기업보다 못한 비효율적 업무 시스템과 직원 복지 및 처우. 일 못(안)하는 20%의 정직원과 그들이 떠넘긴 책임과 업무+갑질을 당하는 80%의 소모품 계약직으로 굴러가는 출자출현기관. 21세기에 아직도 이런곳이 있다는 것이 신기한 조직.': '56',
 '다양한경험을 할수있고 월급이 많지만 힘들어요. 하지만 어떤부분에 있어선편해요': '56',
 '사내정치질이 아주중요한 곳 휩쓸리면 끝': '56',
 '계

In [619]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [620]:
okt=Okt()
text1_noun=[]
for i in range(len(text1)):
    text1_noun.append(okt.nouns(text1[i]))
    

In [621]:
print(text1_noun[:10])

[['병사', '자유도', '부사관', '기준', '육군', '보병', '장교', '업무', '장교', '기준', '생각', '병사', '병사', '기준', '조금', '빡쌔', '근무', '수', '곳', '곳'], ['사서', '로써', '보람', '수', '그', '외', '모든', '것', '곳'], ['대대손손', '척', '진', '적', '입사', '해도', '도시락', '기관', '만약', '내', '누군가', '가스', '팅', '것', '추천', '회사', '개', '쌍욕', '하루', '종일', '타격', '수준', '무', '감정', '추천', '회사', '나', '인간', '임', '포기', '그', '존엄성', '바닥', '보고', '분', '나', '추천'], ['멘토', '일', '개', '봄', '임금', '임원진', '멘토', '관심'], ['조직', '구조', '상', '여러가지', '한계', '점', '다소', '아쉬움'], ['신생', '기관', '체계', '장점', '지자', '단점', '술', '게', '악용', '얼마', '악용'], ['꿈', '희망이', '추천', '월급', '루팡', '곳', '자기개발', '거나', '스스로', '다른', '기관', '이직', '적응'], ['누군가', '원수', '추천', '전형', '회사', '임'], ['별', '개도', '정말', '최악', '직장', '출퇴근길', '퇴사', '생각', '곳'], ['줄', '정리', '공공기관']]


In [622]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  61.307


In [623]:
sent_num=0
text1_sent=[]
for i in range(len(text1)):
    text1_pos=okt.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)
        if n[1] in dict_pos: 
            pos_dict=True
        else: 
            pos_dict=False
        n.insert(3, pos_dict)
        if n[1] in dict_neg: 
            neg_dict=True
        else: 
            neg_dict=False
        n.insert(4, neg_dict)
        
        
        if float(score1[(sent_num-1)]) > avr:
            n.insert(5, '100')
        else: 
            n.insert(5, '0')
        
        #n.insert(5, score1[(sent_num-1)])
        
        text1_sent.append(n)
        

pprint.pprint(text1_sent)

[['Sentence: 1', '병사', 'Noun', False, False, '0'],
 ['Sentence: 1', '보다', 'Josa', False, False, '0'],
 ['Sentence: 1', '높은', 'Adjective', True, False, '0'],
 ['Sentence: 1', '자유도', 'Noun', False, False, '0'],
 ['Sentence: 1', '(', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '부사관', 'Noun', False, False, '0'],
 ['Sentence: 1', '기준', 'Noun', False, False, '0'],
 ['Sentence: 1', '),', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '육군', 'Noun', False, False, '0'],
 ['Sentence: 1', '보병', 'Noun', False, False, '0'],
 ['Sentence: 1', '장교', 'Noun', False, False, '0'],
 ['Sentence: 1', '보다', 'Josa', False, False, '0'],
 ['Sentence: 1', '원활한', 'Adjective', False, False, '0'],
 ['Sentence: 1', '업무', 'Noun', False, False, '0'],
 ['Sentence: 1', '(', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '장교', 'Noun', False, False, '0'],
 ['Sentence: 1', '기준', 'Noun', False, False, '0'],
 ['Sentence: 1', '),', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '생각', 'Noun', False, False,

 ['Sentence: 51', '.', 'Punctuation', False, False, '100'],
 ['Sentence: 51', '계약', 'Noun', False, False, '100'],
 ['Sentence: 51', '직', 'Noun', False, False, '100'],
 ['Sentence: 51', '이든', 'Josa', False, False, '100'],
 ['Sentence: 51', '공무원', 'Noun', False, False, '100'],
 ['Sentence: 51', '이든', 'Josa', False, False, '100'],
 ['Sentence: 51', '다', 'Adverb', False, False, '100'],
 ['Sentence: 51', '수평', 'Noun', False, False, '100'],
 ['Sentence: 51', '적', 'Suffix', False, False, '100'],
 ['Sentence: 51', '으로', 'Josa', False, False, '100'],
 ['Sentence: 51', '대', 'Modifier', False, False, '100'],
 ['Sentence: 51', '해주', 'Noun', False, False, '100'],
 ['Sentence: 51', '고', 'Josa', False, False, '100'],
 ['Sentence: 51', '마음', 'Noun', False, False, '100'],
 ['Sentence: 51', '잘맞는', 'Adjective', False, False, '100'],
 ['Sentence: 51', '직원', 'Noun', False, False, '100'],
 ['Sentence: 51', '이', 'Josa', False, False, '100'],
 ['Sentence: 51', '있으면', 'Adjective', False, False, '100'],
 ['Sent

 ['Sentence: 103', '있다', 'Adjective', False, False, '0'],
 ['Sentence: 103', '.', 'Punctuation', False, False, '0'],
 ['Sentence: 103', '특히', 'Adverb', False, False, '0'],
 ['Sentence: 103', '지역', 'Noun', False, False, '0'],
 ['Sentence: 103', '마다', 'Josa', False, False, '0'],
 ['Sentence: 103', '다르게', 'Adjective', False, False, '0'],
 ['Sentence: 103', '경험', 'Noun', False, False, '0'],
 ['Sentence: 103', '할', 'Verb', False, False, '0'],
 ['Sentence: 103', '수', 'Noun', False, False, '0'],
 ['Sentence: 103', '잇는듯', 'Verb', False, False, '0'],
 ['Sentence: 104', '사람', 'Noun', False, False, '0'],
 ['Sentence: 104', '들', 'Suffix', False, False, '0'],
 ['Sentence: 104', '은', 'Josa', False, False, '0'],
 ['Sentence: 104', '대체로', 'Noun', False, False, '0'],
 ['Sentence: 104', '괜찮지만', 'Adjective', False, False, '0'],
 ['Sentence: 104', '위치', 'Noun', False, False, '0'],
 ['Sentence: 104', '나', 'Josa', False, False, '0'],
 ['Sentence: 104', '민원', 'Noun', False, False, '0'],
 ['Sentence: 104', '인

 ['Sentence: 166', '업무', 'Noun', False, False, '100'],
 ['Sentence: 166', '가', 'Josa', False, False, '100'],
 ['Sentence: 166', '과', 'Noun', False, False, '100'],
 ['Sentence: 166', '다', 'Josa', False, False, '100'],
 ['Sentence: 166', '함', 'Noun', False, False, '100'],
 ['Sentence: 167', '종사', 'Noun', False, False, '100'],
 ['Sentence: 167', '자', 'Suffix', False, False, '100'],
 ['Sentence: 167', '가', 'Josa', False, False, '100'],
 ['Sentence: 167', '자기', 'Noun', False, False, '100'],
 ['Sentence: 167', '커리어', 'Noun', False, False, '100'],
 ['Sentence: 167', '쌓고', 'Verb', False, False, '100'],
 ['Sentence: 167', '연', 'Modifier', False, False, '100'],
 ['Sentence: 167', '구', 'Modifier', False, False, '100'],
 ['Sentence: 167', '활동', 'Noun', False, False, '100'],
 ['Sentence: 167', '하고', 'Josa', False, False, '100'],
 ['Sentence: 167', '좋은', 'Adjective', True, False, '100'],
 ['Sentence: 167', '것', 'Noun', False, False, '100'],
 ['Sentence: 167', '같다', 'Adjective', False, False, '100'],

 ['Sentence: 215', '.', 'Punctuation', False, False, '100'],
 ['Sentence: 215', '조', 'Modifier', False, False, '100'],
 ['Sentence: 215', '사관', 'Noun', False, False, '100'],
 ['Sentence: 215', '들', 'Suffix', False, False, '100'],
 ['Sentence: 215', '친절함', 'Adjective', False, False, '100'],
 ['Sentence: 215', '.', 'Punctuation', False, False, '100'],
 ['Sentence: 216', '세종시', 'Noun', False, False, '100'],
 ['Sentence: 216', '에서', 'Josa', False, False, '100'],
 ['Sentence: 216', '활동', 'Noun', False, False, '100'],
 ['Sentence: 216', '하기', 'Verb', False, False, '100'],
 ['Sentence: 216', '좋은', 'Adjective', True, False, '100'],
 ['Sentence: 216', '기업', 'Noun', False, False, '100'],
 ['Sentence: 216', '으로', 'Josa', False, False, '100'],
 ['Sentence: 216', '기업', 'Noun', False, False, '100'],
 ['Sentence: 216', '지', 'Josa', False, False, '100'],
 ['Sentence: 216', '원', 'Modifier', False, False, '100'],
 ['Sentence: 216', '기관', 'Noun', False, False, '100'],
 ['Sentence: 216', '이고', 'Josa', Fal

 ['Sentence: 269', '좋은', 'Adjective', True, False, '100'],
 ['Sentence: 269', '곳', 'Noun', False, False, '100'],
 ['Sentence: 269', '.', 'Punctuation', False, False, '100'],
 ['Sentence: 269', '정규직', 'Noun', False, False, '100'],
 ['Sentence: 269', '으로', 'Josa', False, False, '100'],
 ['Sentence: 269', '다니', 'Verb', False, False, '100'],
 ['Sentence: 269', '기', 'Noun', False, False, '100'],
 ['Sentence: 269', '엔', 'Josa', False, False, '100'],
 ['Sentence: 269', '다소', 'Noun', False, False, '100'],
 ['Sentence: 269', '피곤할수도', 'Adjective', False, False, '100'],
 ['Sentence: 270', '필요한', 'Adjective', False, False, '100'],
 ['Sentence: 270', '부분', 'Noun', False, False, '100'],
 ['Sentence: 270', '잘', 'Verb', True, False, '100'],
 ['Sentence: 270', '지원', 'Noun', False, False, '100'],
 ['Sentence: 270', '해주고', 'Verb', False, False, '100'],
 ['Sentence: 270', '정해진', 'Noun', False, False, '100'],
 ['Sentence: 270', '일만', 'Noun', False, False, '100'],
 ['Sentence: 270', '하면', 'Verb', False, Fal

In [624]:
text1_sent

[['Sentence: 1', '병사', 'Noun', False, False, '0'],
 ['Sentence: 1', '보다', 'Josa', False, False, '0'],
 ['Sentence: 1', '높은', 'Adjective', True, False, '0'],
 ['Sentence: 1', '자유도', 'Noun', False, False, '0'],
 ['Sentence: 1', '(', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '부사관', 'Noun', False, False, '0'],
 ['Sentence: 1', '기준', 'Noun', False, False, '0'],
 ['Sentence: 1', '),', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '육군', 'Noun', False, False, '0'],
 ['Sentence: 1', '보병', 'Noun', False, False, '0'],
 ['Sentence: 1', '장교', 'Noun', False, False, '0'],
 ['Sentence: 1', '보다', 'Josa', False, False, '0'],
 ['Sentence: 1', '원활한', 'Adjective', False, False, '0'],
 ['Sentence: 1', '업무', 'Noun', False, False, '0'],
 ['Sentence: 1', '(', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '장교', 'Noun', False, False, '0'],
 ['Sentence: 1', '기준', 'Noun', False, False, '0'],
 ['Sentence: 1', '),', 'Punctuation', False, False, '0'],
 ['Sentence: 1', '생각', 'Noun', False, False,

In [625]:
dict_pos

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 4863',
 '극성 : 긍정',
 '',
 '(-;',
 '(^^)',
 '(^-^)',
 '(^^*',
 '(^_^)',
 '(^o^)',
 '*^^*',
 '/^o^\\',
 ':(',
 ":'-(",
 ':-(',
 ':-)',
 ':|',
 ';',
 ';)',
 ';-)',
 '^///^',
 '^^',
 '^^//',
 '<3',
 '=)',
 'XD',
 'ㄱㅅ',
 'ㄱㅇㄷ',
 '가격이 싸다',
 '가까이 사귀어',
 '가까이하다',
 '가다듬어',
 '가다듬어 수습하는',
 '가다듬어 수습하다',
 '가다듬어 정하다',
 '가당히',
 '가뜬하게',
 '가라앉혀 바로잡다',
 '가르치며',
 '가르침',
 '가벼운 마음으로',
 '가볍게 웃

In [626]:
dict_neg

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 9826',
 '극성 : 부정',
 '',
 '가난',
 '가난뱅이',
 '가난살이',
 '가난살이하다',
 '가난설음',
 '가난에',
 '가난에 쪼들려서',
 '가난하게',
 '가난하고',
 '가난하고 어렵다',
 '가난하고 천한',
 '가난하다',
 '가난하여',
 '가난한',
 '가난한 데다가',
 '가난한 사람을',
 '가난히',
 '가년스럽다',
 '가녕스럽다',
 '가누지 못하고',
 '가누지 못하다',
 '가누지 못할',
 '가누지 못할 정도로',
 '가누지 못함',
 '가눌 수 없게',
 '가눌 수 없을',
 '가눌 수 없이',
 '가당찮다',
 '가당찮이',
 '가두거나 해치거나',
 '가망이 없게',
 '가슴 아파하다',
 '가슴 아파함',

In [627]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [628]:
data.columns = ['Sentence #', 'Word', 'POS', 'dict_pos', 'dict_neg', 'score']

In [629]:
data.head(10)

,Sentence #,Word,POS,dict_pos,dict_neg,score
0,Sentence: 1,병사,Noun,False,False,0
1,Sentence: 1,보다,Josa,False,False,0
2,Sentence: 1,높은,Adjective,True,False,0
3,Sentence: 1,자유도,Noun,False,False,0
4,Sentence: 1,(,Punctuation,False,False,0
5,Sentence: 1,부사관,Noun,False,False,0
6,Sentence: 1,기준,Noun,False,False,0
7,Sentence: 1,"),",Punctuation,False,False,0
8,Sentence: 1,육군,Noun,False,False,0
9,Sentence: 1,보병,Noun,False,False,0


In [630]:
data.tail(10)

,Sentence #,Word,POS,dict_pos,dict_neg,score
5608,Sentence: 299,기업,Noun,False,False,100
5609,Sentence: 299,.,Punctuation,False,False,100
5610,Sentence: 300,희망,Noun,True,False,100
5611,Sentence: 300,일자리,Noun,False,False,100
5612,Sentence: 300,로,Josa,False,False,100
5613,Sentence: 300,근무,Noun,False,False,100
5614,Sentence: 300,중,Noun,False,False,100
5615,Sentence: 300,인데,Josa,False,False,100
5616,Sentence: 300,좋음,Adjective,True,False,100
5617,Sentence: 300,~~~~~~~~,Punctuation,False,False,100


In [631]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
dict_pos      0
dict_neg      0
score         0
dtype: int64

In [632]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1787

In [633]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p, dp, dn, s) for w, p, dp, dn, s in zip(s["Word"].values.tolist(),
                                                                     s["POS"].values.tolist(),
                                                                     s["dict_pos"].values.tolist(),
                                                                     s["dict_neg"].values.tolist(),
                                                                     s["score"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
        print(self.grouped)
        self.sentences
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [634]:
# 전체 문장 추출
getter = SentenceGetter(data)

Sentence #
Sentence: 1      [(병사, Noun, False, False, 0), (보다, Josa, False...
Sentence: 10     [(한, Determiner, False, False, 0), (줄, Noun, F...
Sentence: 100    [(청소년, Noun, False, False, 0), (업계, Noun, Fals...
Sentence: 101    [(보람, Noun, True, False, 0), (차게, Verb, False,...
Sentence: 102    [(일, Noun, False, False, 0), (은, Josa, False, ...
                                       ...                        
Sentence: 95     [(준, Noun, False, False, 0), (공공기관, Noun, Fals...
Sentence: 96     [(학교, Noun, False, False, 0), (라는, Josa, False...
Sentence: 97     [(공기업, Noun, False, False, 0), (을, Josa, False...
Sentence: 98     [(퇴근, Noun, False, False, 0), (도, Josa, False,...
Sentence: 99     [(점차, Noun, False, False, 0), (체계, Noun, False...
Length: 300, dtype: object


In [635]:
# 문장 하나
sent = getter.get_next()

In [636]:
# 추출한 문장 확인(적용 결과)
print(sent)

[('병사', 'Noun', False, False, '0'), ('보다', 'Josa', False, False, '0'), ('높은', 'Adjective', True, False, '0'), ('자유도', 'Noun', False, False, '0'), ('(', 'Punctuation', False, False, '0'), ('부사관', 'Noun', False, False, '0'), ('기준', 'Noun', False, False, '0'), ('),', 'Punctuation', False, False, '0'), ('육군', 'Noun', False, False, '0'), ('보병', 'Noun', False, False, '0'), ('장교', 'Noun', False, False, '0'), ('보다', 'Josa', False, False, '0'), ('원활한', 'Adjective', False, False, '0'), ('업무', 'Noun', False, False, '0'), ('(', 'Punctuation', False, False, '0'), ('장교', 'Noun', False, False, '0'), ('기준', 'Noun', False, False, '0'), ('),', 'Punctuation', False, False, '0'), ('생각', 'Noun', False, False, '0'), ('보다', 'Josa', False, False, '0'), ('편한', 'Adjective', True, False, '0'), ('병사', 'Noun', False, False, '0'), ('(', 'Punctuation', False, False, '0'), ('병사', 'Noun', False, False, '0'), ('기준', 'Noun', False, False, '0'), (')', 'Punctuation', False, False, '0'), ('\r ', 'Foreign', False, False, '0

In [637]:
# 전체 문장
sentences = getter.sentences

In [638]:
print(len(sentences)) # 개수 확인

# 문장 하나 확인
print(sentences[58])
print(sentences[:2]) # list의 list 형태로 여러 개 표시. 

# 각각 word, pos, tag, pare로 만들어서 결과 확인

300
[('지역', 'Noun', False, False, '100'), ('특성', 'Noun', False, False, '100'), ('인', 'Josa', False, False, '100'), ('지', 'Verb', False, False, '100'), ('굉장히', 'Adjective', False, False, '100'), ('근무', 'Noun', False, False, '100'), ('가', 'Josa', False, False, '100'), ('여유', 'Noun', True, False, '100'), ('로움', 'Noun', False, False, '100'), ('.', 'Punctuation', False, False, '100')]
[[('병사', 'Noun', False, False, '0'), ('보다', 'Josa', False, False, '0'), ('높은', 'Adjective', True, False, '0'), ('자유도', 'Noun', False, False, '0'), ('(', 'Punctuation', False, False, '0'), ('부사관', 'Noun', False, False, '0'), ('기준', 'Noun', False, False, '0'), ('),', 'Punctuation', False, False, '0'), ('육군', 'Noun', False, False, '0'), ('보병', 'Noun', False, False, '0'), ('장교', 'Noun', False, False, '0'), ('보다', 'Josa', False, False, '0'), ('원활한', 'Adjective', False, False, '0'), ('업무', 'Noun', False, False, '0'), ('(', 'Punctuation', False, False, '0'), ('장교', 'Noun', False, False, '0'), ('기준', 'Noun', False, Fa

In [639]:
# organization 정하여서 특징 정하기
# dict_org = ["특정한 키워드!", "정","하","기"] 
# 캐릭터 개수
# '개월' 관련 키워드
# dict_time = ["월", "달", "년", "일", "분", "시간", "초", "시", "날짜", "날", "오늘", "내일", "모레", "어제", "현재", "과거", "미래"]


# 여러가지 시도해보기!


# feature1: 

# 단어를 features화
# "feat의 이름" : "value"
# 하나의 단어에서 다양한 feature(단어, POS tag, lables 등) 확인을 위해 문장 전체를 넣어줌. 

def word2features(sent, i):
    word = sent[i][0]   # index0의 단어를 가져옴
    postag = sent[i][1] # index1의 단어를 가져옴
    dict_pos_tag = sent[i][2]
    dict_neg_tag = sent[i][3]
    score = sent[i][4]

# postag 유지
# postag2 = kkma.pos(word)
# 
    features = {
        'bias': 1.0,
        'postag': postag,
        'dict_pos' : dict_pos_tag,
        'dict_neg' : dict_neg_tag,
        'word.len' : len(word)
#'dict_time': word.lower() in dict_time # -> true or false로
# character 개수 
# 'word.len' : len(word)
# 'word.len, num' : 
    }
    
    # 직전의 단어에 대해서도 똑같이 적용
    if i > 0:
        word1 = sent[i-1][0] # i-1: 직전의 단어
        postag1 = sent[i-1][1]
        # update: 기존에서 추가! (하여 새로운 dictionary를 만드는 결과)
        features.update({
            '-1:postag': postag,
            '-1:dict_pos' : dict_pos_tag,
            '-1:dict_neg' : dict_neg_tag,
            '-1:word.len' : len(word)
        })
    #else:
    #    features['BOS'] = True # BOS: Beginning Of Sentence

    # 다음 단어에 대해서도 똑같이 처리
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag,
            '+1:dict_pos' : dict_pos_tag,
            '+1:dict_neg' : dict_neg_tag,
            '+1:word.len' : len(word)
        })
    
    #else:
    #    features['EOS'] = True  # EOS: end of sentence
        
    # bi-그램에 대해서도 똑같이 처리
    
    return features

# sentence 전체에 적용하기 위한 함수
def sent2features(sent):
    for i in range(len(sent)):
        print(word2features(sent, i))
    return [word2features(sent, i) for i in range(len(sent))] # 0~i(index의 수)까지 적용 -> sentence에 적용
# == features = [word2features(sent, i) for i in range(len(sent))]
# return features

def sent2score(sent): # sent를 읽어서 for문을 돌림. label만 갖고 옴
    return [score for token, postag, dict_pos_tag, dict_neg_tag, score in sent]
# 지금은 3개가 tuple로 있음.

def sent2tokens(sent):
    return [token for token, postag, dict_pos_tag, dict_neg_tag, score in sent]

In [640]:
sent2score(sent)

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [641]:
print(sent2features(sent))

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Adjective', '-1:dict_pos': True, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Adjective', '+1:dict_pos': True, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Punctuation', 'dict_pos': False, 'dict_neg

In [642]:
X = [sent2features(s) for s in sentences] # sentence마다 feature로 바뀌고
y = [sent2score(s) for s in sentences] # sentence마다 score로 바뀜

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Adjective', '-1:dict_pos': True, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Adjective', '+1:dict_pos': True, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Punctuation', 'dict_pos': False, 'dict_neg

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len

{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Adjective', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Adjective', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Adjective', '-1:dict_pos': True, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Adjective', '+1:dict_pos': True, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Punctuation', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Punctuation', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Punctuation', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': 

{'bias': 1.0, 'postag': 'Punctuation', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Punctuation', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Punctuation', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Adjective', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Adjective', '+1:dict_pos': False, 

{'bias': 1.0, 'postag': 'Adjective', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Adjective', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 4, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 4, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 4}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 3, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': True, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': True, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 

{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len':

{'bias': 1.0, 'postag': 'Verb', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Verb', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Verb', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Noun', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Noun', '-1:dict_pos': True, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Noun', '+1:dict_pos': True, '+1:dict_neg': False, '+1:word.len': 2}
{'bias': 1.0, 'postag': 'Josa', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'Josa', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 1, '+1:postag': 'Josa', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 1}
{'bias': 1.0, 'postag': 'Verb', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'Verb', '-1:dict_pos': False, '-1:dict_neg': False, '-1:word.len': 2, '+1:postag': 'Verb', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 2}

In [643]:
y

[['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0', '0', '0', '0'

In [644]:
print(len(X))
print(len(y))
print(len(sentences))

300
300
300


In [645]:
#10번째 sentence의 정보 
print(len(X[10])) 
print(len(y[10]))
print(sentences[10])
print(y[10])
print(X[10])

19
19
[('아무리', 'Adverb', False, False, '0'), ('변', 'Noun', False, False, '0'), ('한다해', 'Noun', False, False, '0'), ('도', 'Josa', False, False, '0'), ('군대', 'Noun', False, False, '0'), ('는', 'Josa', False, False, '0'), ('군대', 'Noun', False, False, '0'), ('..', 'Punctuation', False, False, '0'), ('위', 'Noun', False, False, '0'), ('에', 'Josa', False, False, '0'), ('분', 'Noun', False, False, '0'), ('들', 'Suffix', False, False, '0'), ('의', 'Josa', False, False, '0'), ('생각', 'Noun', False, False, '0'), ('은', 'Josa', False, False, '0'), ('아직', 'Adverb', False, False, '0'), ('옛날', 'Noun', False, False, '0'), ('이', 'Determiner', False, False, '0'), ('더', 'Noun', False, False, '0')]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[{'bias': 1.0, 'postag': 'Adverb', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '+1:postag': 'Adverb', '+1:dict_pos': False, '+1:dict_neg': False, '+1:word.len': 3}, {'bias': 1.0, 'postag': 'Noun', 'dict_pos': Fals

In [646]:
# 문장 하나 확인
# 각 단어의 features가 dictionary로 표현되고 이를 요소로 하는 리스트
X[0]
# 문장은 단어로 이루어져 있고, 각 단어의 features 확인 가능
# 첫 번째 단어는 앞 단어에 대한 정보는 없고 BOS = true

[{'bias': 1.0,
  'postag': 'Noun',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 2,
  '+1:postag': 'Noun',
  '+1:dict_pos': False,
  '+1:dict_neg': False,
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'Josa',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 2,
  '-1:postag': 'Josa',
  '-1:dict_pos': False,
  '-1:dict_neg': False,
  '-1:word.len': 2,
  '+1:postag': 'Josa',
  '+1:dict_pos': False,
  '+1:dict_neg': False,
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'Adjective',
  'dict_pos': True,
  'dict_neg': False,
  'word.len': 2,
  '-1:postag': 'Adjective',
  '-1:dict_pos': True,
  '-1:dict_neg': False,
  '-1:word.len': 2,
  '+1:postag': 'Adjective',
  '+1:dict_pos': True,
  '+1:dict_neg': False,
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'Noun',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 3,
  '-1:postag': 'Noun',
  '-1:dict_pos': False,
  '-1:dict_neg': False,
  '-1:word.len': 3,
  '+1:postag': 'Noun',
  '+1:dict_pos': False,
  '+1:dict_neg':

In [647]:
print(len(X), len(X)*0.7) #train-test split (70%, 30%)
print(type(X))

300 210.0
<class 'list'>


In [648]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [649]:

# Training
crf.fit(X[:300], y[:300])

c:\users\sj\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=10, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [655]:
# cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5) # cv=n 나중에 주기

In [656]:
# prediction
y_test_pred = crf.predict(X[300:])

In [657]:
# report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

           0       0.45      0.47      0.46      2916
         100       0.41      0.39      0.40      2702

    accuracy                           0.43      5618
   macro avg       0.43      0.43      0.43      5618
weighted avg       0.43      0.43      0.43      5618



In [658]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
# eli5: weight visualization
eli5.show_weights(crf, top=30) # -> 중요한 것만 확인

In [659]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(X)

NameError: name 'CountVectorizer' is not defined